In [163]:
#!/usr/bin/env python

"""
takes all rows for the 50 most-posted-in groups, removes the wierd users, and outputs subset.
matches users in this subset to a uniqe id.
ONLY ASIGNS ID WHEN LAST LOGIN IS THE SAME
Takes the EP_data.csv file.
"""

import pandas as pd
import numpy as np

datafile = pd.read_csv('EP_data.csv')

#get a list of the 50 groups with the most posts
group_count = pd.Series(datafile["gid"])
top50 = group_count.value_counts()[0:50]
print (top50)

1431      8257
193494    6828
117528    5085
2510      4108
154884    3388
109       2851
8726      2610
4603      2133
233       2109
1009      1905
42844     1859
17036     1612
809       1335
102852    1303
23323     1287
950       1246
335       1223
233190    1189
332       1185
11525     1102
2108      1101
426065    1081
2534      1056
1843      1016
649        985
51387      985
763        956
1261       931
2475       927
984        924
76430      919
43748      912
147547     912
1340       870
646363     868
8083       857
887        735
2334       672
165        662
1728       611
5382       603
15349      602
68         598
4527       597
25529      596
828259     584
974071     578
364326     562
166260     543
5577       539
dtype: int64


In [155]:
sumtop50 = top50.sum()
print ('there are %d total posts in the top 50 groups') % sumtop50

there are 76397 total posts in the top 50 groups


In [157]:
groups = []
for index, row in top50.iteritems():
    groups.append(index)
#print 'The top 50 groups are:',groups

In [158]:
#make a smaller dataset containing only entries with top group ids
subset_dirty = datafile[datafile['gid'].isin(groups)]
print len(subset_dirty), 'should equal the number of posts calculated above:', sumtop50
#clean subset, removing entries by suspected moderator
subset = subset_dirty[subset_dirty['num_entries']!=65535]
print 'after removing fake user, the list is this long:',len(subset)

76397 should equal the number of posts calculated above: 76397
after removing fake user, the list is this long: 71369


In [161]:
#add user id to the subset

members = pd.read_csv('members.csv')
#print members.columns
#print subset.columns

IDsubset = pd.merge(subset, members, how='inner', on=['created','birthdate','gender','last_login'])

#print IDsubset.head(n = 5)
print subset.shape
print members.shape
print IDsubset.shape

(71369, 16)
(972845, 5)
(23177, 17)


In [162]:
#if we think we need to take more users, as in guessing user id based upon
#number of users with same 'created' 'bday' and 'gender' but not the same
#'last login', then we should go back and add these
#I didn't do it the previous way because it was too slow for my computer.


#get a list of the 50 groups with the most posts
g_count = pd.Series(IDsubset["gid"])
remainders = g_count.value_counts()[0:50]
print (remainders)

117528    3056
154884    1460
1431      1105
109       1066
193494    1047
4603      1023
233        991
2510       933
2475       677
42844      554
335        497
984        486
8726       409
1843       405
1009       405
332        361
809        352
17036      340
364326     339
950        337
887        334
165        331
646363     325
2108       324
43748      323
763        320
1340       319
102852     316
649        311
68         311
233190     290
23323      281
828259     279
166260     276
25529      276
2334       267
5577       263
2534       250
76430      235
11525      228
147547     211
5382       200
1261       189
8083       186
974071     178
15349      143
51387      133
1728       128
4527       107
dtype: int64


In [164]:
IDsubset.to_csv('topgroups_ids.csv')